# Test stuff for PFN model

Classify pions, photons, and **scalars**

In [2]:
# Add import paths
import sys
sys.path.append("..")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import datetime as dt
from utils import print_gpu_memory

# Make tensorflow not use too much memory
print(f"Importing tensorflow...")
import tensorflow as tf
print(f"Enabling memory growth...")
gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Importing tensorflow...
Enabling memory growth...


In [3]:
import yaml
with open("../config.yaml") as fin:
    model_dir = yaml.safe_load(fin)["model_dir"]

In [4]:
# Get data
from data import get_data
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = get_data("processed/scalars/all_jets_point_cloud.npz")

print(X_train.shape, Y_train.shape)

/usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/data
(240000, 960, 4) (240000, 3)


In [5]:
# Build model
from model import PFN
Phi_sizes = [128, 128, 128, 128, 64, 64, 64]
F_sizes = [128, 128, 128, 128, 64, 64, 64]

_, n_particles, n_features = X_train.shape
model = PFN(
    n_features=n_features,
    n_particles=n_particles,
    n_outputs=Y_train.shape[1],
    Phi_sizes=Phi_sizes,
    F_sizes=F_sizes
)

In [6]:
from collections import defaultdict
history = defaultdict(list)

In [7]:
# Utility functions for training
from train_model import train_model
import yaml

def train_iteration(lr, epochs):
    fit_history = train_model(
        model=model, 
        data=(X_train, X_val, Y_train, Y_val),
        lr=lr,
        epochs=epochs
    )
    print(f"\nSaving history...")
    for key in ["loss", "val_loss", "accuracy", "val_accuracy"]:
        history[key].extend(fit_history.history[key])
    return fit_history

def save_model(name):
    cur_date = dt.datetime.now().strftime("%Y-%m-%d")
    model.save(f"{model_dir}/{name}_{cur_date}")

In [8]:
print_gpu_memory()

GPU memory usage
  current: 556.75 KB
  peak:    618.25 KB


In [ ]:
print(f"=== Training [{dt.datetime.now()}] ===")
train_iteration(lr=2e-4, epochs=30)

=== Training [2023-06-08 01:18:40.171011] ===
Epoch 1/30
 112/2400 [>.............................] - ETA: 41s - loss: 5.2985 - accuracy: 0.3346

In [ ]:
print(f"=== Training [{dt.datetime.now()}] ===")
train_iteration(lr=2e-5, epochs=30)

In [ ]:
print(f"=== Training [{dt.datetime.now()}] ===")
train_iteration(lr=2e-6, epochs=30)

In [ ]:
save_model("7_feature_95pc")

### Evaluate model

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
ax1, ax2 = axs

ax1.plot(history["loss"])
ax1.plot(history["val_loss"])
ax1.legend(["loss", "val_loss"])
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss");

ax2.plot(history["accuracy"])
ax2.plot(history["val_accuracy"])
ax2.legend(["loss", "val_accuracy"])
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy");

In [ ]:
# Test model
from test_model import test_model, plot_cm
test_accuracy, cm = test_model(model, (X_test, Y_test))
print(f"Testing accuracy: {test_accuracy}")

plot_cm(cm, ["pion", "photon", "axion1"])